In [3]:
import os
import pandas as pd
import numpy as np
import warnings
from collections import defaultdict
from datetime import datetime, date, timedelta
import calendar
import traceback

dataset = pd.read_excel('Contracts_Dataset.xlsx', dtype=str)


In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time

# Get all the rows that have Tender Links
tender_link_rows = dataset[dataset['TenderLink'].notna()]
print(tender_link_rows.shape[0])
print(tender_link_rows['Reference Number'].nunique())
print(tender_link_rows['TenderLink'].nunique())

base_download_dir = r"C:\Users\aanand\OneDrive - Intel Corporation\Desktop\Misc\TenderFiles"
unique_tender_links = tender_link_rows['TenderLink'].unique()


counter = 0

# Wait for the file to appear in the download directory
timeout = 60  # Set a reasonable timeout in seconds
downloaded = False

for one_tender_link in unique_tender_links:
    try:
        
        # Find the matching reference number - will be used to create folders
        reference_number = tender_link_rows[tender_link_rows['TenderLink'] == one_tender_link]['Reference Number'].iloc[0]
        #download_dir = os.path.join(base_download_dir, reference_number)
        download_dir = base_download_dir

        chrome_options = Options()
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-gpu")
        
        prefs = {"download.default_directory": download_dir}
        chrome_options.add_experimental_option("prefs", prefs)

        driver = webdriver.Chrome(options=chrome_options)

        driver.get(one_tender_link)
        
        link_text = 'Download Now'
        link_element = driver.find_element(By.LINK_TEXT, link_text)

        # Click on the <a> element
        link_element.click()

        button = driver.find_element(By.CLASS_NAME, "BUTTON")
        button.click()


        button = driver.find_element(By.CLASS_NAME, "SUBMIT")

        start_time = time.time()
        button.click()
        
        time.sleep(2)
        
        # Wait for the file to download (adjust the time as needed)
        while time.time() - start_time < timeout:
            files = os.listdir(download_dir)  # Replace with the actual download directory
            if any(file.endswith('.crdownload') for file in files):
                time.sleep(1)  # Wait a bit before checking again
            else:
                break
        
        counter += 1
        if counter % 100 == 0:
            print(counter)

    except:
        print(one_tender_link)
        continue
    finally:
        driver.quit()


6296
3326
3321
100
